In [ ]:
import pandas as pd
import re
from openpyxl import Workbook

def month_number_to_name(month):
    """Convert month number to abbreviated month name."""
    return ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][month - 1]

def get_last_three_months(current_month, current_year):
    """Return last three months data."""
    months = []
    for i in range(3):
        month = (current_month - i) % 12
        year = current_year - (1 if month == 0 else 0)
        month = month if month != 0 else 12
        months.append((month, year))
    return months

def get_monthly_data(sheet, current_month, current_year):
    """Return the columns corresponding to the last three months."""
    month_columns = []
    last_three_months = get_last_three_months(current_month, current_year)

    for col in sheet.columns:
        if re.match(r'M\d{2}', col):
            month_num = int(col[1:3])
            year = int(col[3:])

            if (month_num, year) in last_three_months:
                month_columns.append(col)

    return month_columns

def get_ytd_data(sheet, current_month):
    """Return sum of Mxx columns up to the current month for YTD calculations."""
    month_columns = []
    for col in sheet.columns:
        if re.match(r'M\d{2}', col):
            month_num = int(col[1:3])
            month_columns.append(col) if month_num <= current_month else None
    return month_columns

def process_paste_sheet(processed_file_path, current_month, current_year):
    # Load the processed file
    xls = pd.ExcelFile(processed_file_path)
    
    # Create a new workbook and the "Paste" sheet
    wb = Workbook()
    ws = wb.active
    ws.title = "Paste"

    # Define the headers for the "Paste" sheet
    headers = ["File Name", "Level 3", "Level 4", "Cost Grouping", "Cost Type", 
               "Finance Region", "Attribute Type", "Period", "Cost", 
               "FTE/Contractor", "Country", "Level 3.5", "Level 4.5", "Entity", 
               "Function", "Mapped Country", "MICA"]
    ws.append(headers)  # Write headers to the first row

    for sheet_name in xls.sheet_names:
        sheet = pd.read_excel(xls, sheet_name=sheet_name)

        # Determine if it's a DC or HC sheet
        cost_type = 'Cost' if 'DC' in sheet_name else 'FTE/Contractor' if 'HC' in sheet_name else None

        # Handle MoM for last three months
        if re.search(r'Actual|Target|Fcst', sheet_name, re.IGNORECASE):
            month_columns = get_last_three_months(current_month, current_year)

            # For each month column, get data
            for month, year in month_columns:
                month_col_name = f'M{month:02d}{year}'
                if month_col_name in sheet.columns:
                    total = sheet[month_col_name].sum()
                    period_value = f"{month_number_to_name(month)}-{str(year)[-2:]}"
                    ws.append([sheet_name, '', '', '', '', '', 'MoM', period_value, total if cost_type == 'Cost' else '', total if cost_type == 'FTE/Contractor' else '', '', '', '', '', '', '', ''])

        # Handle other attribute types
        if cost_type:
            if 'Target' in sheet_name:
                attribute_type = 'FY Target'
                fy_column = f'FY {current_year}'
                if fy_column in sheet.columns:
                    period_value = f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}"
                    ws.append([sheet_name, '', '', '', '', '', attribute_type, period_value, '', '', '', '', '', '', '', ''])

            elif 'Fcst' in sheet_name:
                attribute_type = 'FY Forecast'
                fy_column = f'FY {current_year}'
                if fy_column in sheet.columns:
                    period_value = f"Dec-{current_year}"
                    ws.append([sheet_name, '', '', '', '', '', attribute_type, period_value, '', '', '', '', '', '', '', ''])

            elif 'Actual' in sheet_name:
                attribute_type = 'YTD Actual'
                month_columns = get_ytd_data(sheet, current_month)
                total = sum(sheet[col] for col in month_columns if col in sheet.columns)

                period_value = f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}"
                ws.append([sheet_name, '', '', '', '', '', attribute_type, period_value, total if cost_type == 'Cost' else '', total if cost_type == 'FTE/Contractor' else '', '', '', '', '', '', ''])

            # Handle Prior Year FY and YTD similarly...
            fy_column_prior = f'FY {current_year - 1}'
            if fy_column_prior in sheet.columns:
                ws.append([sheet_name, '', '', '', '', '', 'Prior Year FY', 'Dec-' + str(current_year - 1), sheet[fy_column_prior].sum(), '', '', '', '', '', '', ''])

            ytd_column_prior = f'YTD {current_year - 1}'
            if ytd_column_prior in sheet.columns:
                ws.append([sheet_name, '', '', '', '', '', 'Prior Year YTD', f"{month_number_to_name(current_month)}-{str(current_year - 1)[-2:]}", sheet[ytd_column_prior].sum(), '', '', '', '', '', '', ''])

    # Save the final workbook
    wb.save('final_output_file.xlsx')

# Example usage
current_month = 6  # June
current_year = 2024
processed_file_path = 'Processed_file.xlsx'  # Update with your file path
process_paste_sheet(processed_file_path, current_month, current_year)
